## BÁO CÁO:<br> PHÂN TÍCH DỮ LIỆU VÀ XÂY DỰNG MÔ HÌNH DỰ ĐOÁN KHÁCH HÀNG ĐĂNG KÝ TIỀN GỬI CÓ KỲ HẠN

Đầu tiên ta import dữ liệu. <br>1 file bank-additional-full.csv dùng để train.<br>1 file bank-additional.csv dùng để test model
<br> Dùng class ProfileReport trong thư viện ydata_profiling để generate ra bản báo cáo dữ liệu. 

In [10]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTEN
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.linear_model import SGDClassifier


In [11]:
data_train = pd.read_csv('bank-additional-full.csv', sep=';')
data_test = pd.read_csv('bank-additional.csv', sep=';')
print(data_train.shape)


(41188, 21)


In [12]:

duplicate_counts = data_train[data_train.duplicated(keep=False)]
print("Số dòng trùng trong full:", duplicate_counts.shape[0])

# Loại bỏ các dòng trùng lặp, chỉ giữ lại bản gốc đầu tiên
data_train = data_train.drop_duplicates(keep='first')
print("Số dòng sau khi loại bỏ trùng lặp:", data_train.shape[0])

# Bước 1: Merge với indicator để đánh dấu dòng trùng
merged = pd.merge(
    data_train, 
    data_test, 
    how="left",  
    indicator=True 
)

# Bước 2: Lọc chỉ dòng KHÔNG trùng (_merge == 'left_only')
data_train_cleaned = merged[merged['_merge'] == 'left_only'].drop(columns='_merge')

print("Số dòng ban đầu:", len(data_train))
print("Số dòng trùng với test:", len(data_train) - len(data_train_cleaned))
print("Số dòng sau khi loại bỏ trùng:", len(data_train_cleaned))



Số dòng trùng trong full: 24
Số dòng sau khi loại bỏ trùng lặp: 41176
Số dòng ban đầu: 41176
Số dòng trùng với test: 4119
Số dòng sau khi loại bỏ trùng: 37057


In [13]:
x_train = data_train_cleaned.drop(columns=['y'])
y_train = data_train_cleaned['y']
x_test = data_test.drop(columns=['y'])
y_test = data_test['y']
print(x_train.shape, y_train.shape)


(37057, 20) (37057,)


In [14]:

target = {'yes' : 1, 'no': 0}
y_train = y_train.map(target)
y_test = y_test.map(target)
print(y_train.value_counts())
print('------------------------')
column_names = x_train.columns
smoten = SMOTEN(random_state=42, sampling_strategy=0.7, k_neighbors=5, n_jobs=-1)
x_train, y_train = smoten.fit_resample(x_train, y_train)
x_train = pd.DataFrame(x_train, columns=column_names)
print(y_train.value_counts())


y
0    32869
1     4188
Name: count, dtype: int64
------------------------


c:\Users\buiqu\anaconda3\Lib\site-packages\imblearn\over_sampling\_smote\base.py:992: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


y
0    32869
1    23008
Name: count, dtype: int64


### Data Preprocessing
<br> Tiến hành xử lý các cột
<br> Chia dữ liệu thành 2 loại
<br>1. Numerical (dạng số): Có thể số nguyên hoặc thực
<br>2. Categofical (Phân loại): gồm Nominal (định danh), Ordinal(thứ bậc), boolen (logic) 

Có 2 cột dạng số đặc biệt là cột month và day_of_week vì dữ liệu có tính chất chu kỳ (cyclical). 
<br>Chúng ta xử dụng một kỹ thuật cyclical_encoding để sử dụng hàm sin(), cos() để ánh xạ giá trị vào không gian 2 chiều (Trên vòng tròn đơn vị)
<br>Có một cột đặc biệt nữa là cột default vì giá trị unknow khá cao (20.87%) nên tôi đề xuất 1 giải pháp là tạo riêng 1 cột default_unknow (1 nếu giá trị unknow, 0 nếu không phải), và thay giá trị unknow trong cột default bằng giá trị mode và sau đó mã hóa nhị phân

In [15]:


month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

day_map = {
    'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5
}

x_train['month'] = x_train['month'].map(month_map)
x_test['month'] = x_test['month'].map(month_map)
x_train['day_of_week'] = x_train['day_of_week'].map(day_map)
x_test['day_of_week'] = x_test['day_of_week'].map(day_map)
def cyclical_encoding(df, col, max_val):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_val)
    return df

x_train = cyclical_encoding(x_train, 'month', 12)
x_train = cyclical_encoding(x_train, 'day_of_week', 5)
x_test = cyclical_encoding(x_test, 'month', 12)
x_test = cyclical_encoding(x_test, 'day_of_week', 5)

x_train = x_train.drop(columns=['month', 'day_of_week'])
x_test = x_test.drop(columns=['month', 'day_of_week'])
#tạo cột mới với tên default_unknown, giá trị là 1 nếu default = unknown, ngược lại là 0 ( gồm yes và no)
x_train['default_unknown'] = (x_train['default'] == 'unknown').astype(int)
x_test['default_unknown'] = (x_test['default'] == 'unknown').astype(int)
#Việc một giá trị là 'unknown' không hẳn chỉ là thiếu dữ liệu, mà bản thân sự thiếu đó cũng có thể mang thông tin quan trọng.
numeric_columns = ['age', 'duration', 'campaign', 'pdays', 'previous', 
                   'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 
                   'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos']
nominal_columns = ['job','marital','contact','poutcome']
binary_columns = ['default','housing','loan', 'default_unknown']
ordinal_columns = ['education']

education_categories = ['illiterate','basic.4y', 'basic.6y', 'basic.9y', 'high.school', 
    'professional.course', 'university.degree'
] 
print(x_train['default'].head())
print(x_train.shape)


0         no
1    unknown
2         no
3         no
4         no
Name: default, dtype: object
(55877, 23)


Tiếp theo tiền xử lý các cột dữ liệu và xử lý các giá trị unknown ở từng cột

In [16]:
#Preprocess for numeric data(dữ liệu số)
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')), # xu li du lieu mat mat
    ('scaler', StandardScaler())  
])

#Preprocess for nominal data (dữ liệu phân loại, không có thứ tự)
# Chẳng hạn như 'job', 'marital', 'contact', 'poutcome'
nominal_pipeline = Pipeline([
    ('imputer',SimpleImputer(missing_values='unknown',strategy='most_frequent')),
    ('encoder',OneHotEncoder())
])


#Preprocess for binary data( dữ liệu nhị phân)
# Chẳng hạn như 'default', 'housing', 'loan', 'default_unknown'
binary_pipeline = Pipeline([
    ('imputer',SimpleImputer(missing_values='unknown',strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

#Preprocess for ordinal data(dữ liệu có thứ tự)
# Chẳng hạn như 'education'
ordinal_pipeline = Pipeline([
    ('imputer',SimpleImputer(missing_values='unknown',strategy='most_frequent')),
    ('encoder',OrdinalEncoder(categories=[education_categories]))
])

Gom tất cả các bước tiền xử lý lại trong một đối tượng preprocessor bằng Class ColumnTransformer


In [17]:
# Preprocess for all data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_columns),
        ('nom', nominal_pipeline, nominal_columns),
        ('bin', binary_pipeline, binary_columns),
        ('ord', ordinal_pipeline, ordinal_columns)
    ]
)

Kiểm tra dữ liệu sau khi xử lý

In [18]:
x_train_processed = preprocessor.fit_transform(x_train, y_train)
x_train_processed_df = pd.DataFrame(x_train_processed)
print(x_train.shape)
print(x_train_processed_df.head())

(55877, 23)
         0         1        2         3         4         5         6   \
0  1.577648 -0.147963 -0.48975  0.321786 -0.428384  0.851423  0.812006   
1  1.666544 -0.567177 -0.48975  0.321786 -0.428384  0.851423  0.812006   
2 -0.111360 -0.278967 -0.48975  0.321786 -0.428384  0.851423  0.812006   
3  0.155325 -0.559691 -0.48975  0.321786 -0.428384  0.851423  0.812006   
4  1.577648  0.024214 -0.48975  0.321786 -0.428384  0.851423  0.812006   

         7         8         9   ...   32   33   34   35   36   37   38   39  \
0  0.765485  0.930235  0.565732  ...  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0   
1  0.765485  0.930235  0.565732  ...  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0   
2  0.765485  0.930235  0.565732  ...  0.0  1.0  0.0  0.0  1.0  1.0  0.0  1.0   
3  0.765485  0.930235  0.565732  ...  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0   
4  0.765485  0.930235  0.565732  ...  0.0  1.0  0.0  1.0  0.0  0.0  1.0  1.0   

    40   41  
0  0.0  1.0  
1  1.0  4.0  
2  0.0  4.0  
3  0.0

Ta có thể thấy số lượng đặc trưng lên đến 41 cột sau khi mã hóa. Điều này làm nguy cơ tăng overfitting và làm chậm quá trình huấn luyện.
<br>Ta có thể xử dụng một kỹ thuật SelectPercentile để lựa chọn các đặc trưng (feature_selection) nó đánh giá mức đôn quan trọng từng đặc trưng bằng cách xử dụng một hàm kiểm định thống kê 

In [19]:
# Pipeline preprocessing and future_selection
preprocessing_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', SelectPercentile(score_func=f_classif, percentile=50))
])
x_train_processed = preprocessing_pipeline.fit_transform(x_train, y_train)
x_test_processed = preprocessing_pipeline.transform(x_test)
print(x_train_processed.shape)

(55877, 21)


Áp dụng GridSearchCV cho LGBMClassifier và Logic Regresstion
<br>Vì theo bài toán ta sẽ ưu tiên tìm kiếm khách hàng tiềm năng có nghĩa là khách hàng say Yes (Class Possitive) nên ta ưu tiên độ bao phủ của class Possitive nên tôi lựa chọn Recall làm điểm đánh giá mô hình

In [ ]:


# Dùng recall scorer an toàn

# Pipeline LightGBM
lgbm_pipeline = Pipeline([
    ('preprocessing_and_selection', preprocessing_pipeline),
    ('classifier', LGBMClassifier(random_state=42))
])

# Grid search tham số
lgbm_param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.01, 0.1, 0.3],
    'classifier__num_leaves': [31, 50, 100],
    'classifier__max_depth': [-1, 10, 20],
}


# GridSearchCV tối ưu recall
lgbm_grid_search = GridSearchCV(
    estimator=lgbm_pipeline,
    param_grid=lgbm_param_grid,
    cv=3,
    scoring= 'recall',
    n_jobs=-1,
    verbose=1,
    error_score='raise'
)


# Huấn luyện mô hình
lgbm_grid_search.fit(x_train, y_train)

# In kết quả
print("LGBMClassifier - Bộ tham số tốt nhất:", lgbm_grid_search.best_params_)
print("LGBMClassifier - Recall tốt nhất trên tập train:", lgbm_grid_search.best_score_)

# Đánh giá trên tập test
y_pred_lgbm = lgbm_grid_search.predict(x_test)
print("LGBMClassifier - Kết quả trên tập test:")
print(classification_report(y_test, y_pred_lgbm))




Fitting 3 folds for each of 81 candidates, totalling 243 fits
[LightGBM] [Info] Number of positive: 23008, number of negative: 32869
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002018 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 646
[LightGBM] [Info] Number of data points in the train set: 55877, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.411762 -> initscore=-0.356688
[LightGBM] [Info] Start training from score -0.356688
LGBMClassifier - Bộ tham số tốt nhất: {'classifier__learning_rate': 0.01, 'classifier__max_depth': -1, 'classifier__n_estimators': 300, 'classifier__num_leaves': 100}
LGBMClassifier - Recall tốt nhất trên tập train: 0.8838198101830468
LGBMClassifier - Kết quả trên tập test:
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      36

In [21]:
from sklearn.linear_model import LogisticRegression
# Pipeline với Logistic Regression
logreg_pipeline = Pipeline([
    ('preprocessing_and_selection', preprocessing_pipeline),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear')) 
])

# Grid search tham số cho LogisticRegression
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10],  # nghịch đảo của regularization strength
    'classifier__penalty': ['l1', 'l2'],
}

# GridSearchCV tối ưu recall
grid_search = GridSearchCV(
    estimator=logreg_pipeline,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Huấn luyện mô hình
grid_search.fit(x_train, y_train)

# Dự đoán và đánh giá
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

print("Best Params:", grid_search.best_params_)
print("Best Score (Recall on CV):", grid_search.best_score_)
print("Classification Report on Test Set:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Params: {'classifier__C': 0.01, 'classifier__penalty': 'l2'}
Best Score (Recall on CV): 0.7854667948970125
Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.97      0.87      0.92      3668
           1       0.42      0.75      0.54       451

    accuracy                           0.86      4119
   macro avg       0.70      0.81      0.73      4119
weighted avg       0.91      0.86      0.88      4119

